# Use Insurance Code Template Notebook Template

## Packages needed to run this notebook if running with inbuilt snowpark 3.8 template
##### ! pip install "snowflake-connector-python[pandas]"
##### ! sudo pip install snowflake-ml-python==1.0.11 -U
##### ! pip install --upgrade snowflake-snowpark-python==1.9.0
##### ! pip install --upgrade xgboost==1.7.3
##### ! pip install --upgrade numpy==1.24.3
##### ! pip install --upgrade pandas==1.5.3
##### ! pip install --upgrade anyio==3.5.0
##### ! pip install --upgrade packaging==23.1
##### ! pip install --upgrade scikit-learn==1.3.0
##### ! pip install --upgrade typing-extensions==4.7.1
##### ! pip install --upgrade cryptography==39.0.0
##### ! pip install --upgrade fsspec==2023.9.2
##### ! pip install fosforio
##### ! pip install fosforml

In [1]:
from snowflake.snowpark import Session
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
# import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
# Pandas Tools
from snowflake.connector.pandas_tools import write_pandas
# Data Science Libs
import numpy as np
import pandas as pd
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
# FosforIO to read from snowflake
from fosforio import snowflake
# FosforML to register Model on FDC
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


# Read data using Snowflake's Snowpark

In [2]:
#Import all snowflake connection details from Template or Project variables.
db_user = os.getenv('Snowflake_user')
db_password =  os.getenv('Snowflake_password')
db_account = os.getenv('Snowflake_Account')
db_database =  os.getenv('Snowflake_Database')
db_role = os.getenv('Snowflake_user')
db_warehouse = os.getenv('Snowflake_Warehouse')
db_schema = os.getenv('Snowflake_Schema')

In [3]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [19]:
session.sql('use warehouse FOSFOR_SOLUTIONS_WH;').collect()
session.sql('use database FDC_Insurance;').collect()
session.sql('use schema FDC_Insurance.PUBLIC;').collect()

df = session.table('FDC_Insurance.PUBLIC.AUTO_INSURANCE_CLAIMS_DATA_PRODUCT')

In [20]:
df_backup = df.to_pandas().copy()

# 1. Claim Amount XGB Snowflake Model Prediction

In [21]:
from joblib import dump, load
filename = "Claims_Snowpark_XGB_Regression.joblib" #Model version 1

In [22]:
# Load the Snowflake's Snowpark XGB model
model = load('model_artifacts/' + filename)

In [23]:
df = df.to_pandas().replace(np.nan, pd.isna)

In [24]:
df["AUTO_YEAR"] = df["AUTO_YEAR"].astype(str)

In [25]:
pred = model.predict(df)
df_backup["CLAIM_AMOUNT_PREDICTION"] = pred["PREDICTION"]

# Merge XGB Model output in a back up dataframe for reference

In [26]:
df_backup.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,POLICE_REPORT_AVAILABLE,TOTAL_CLAIM_AMOUNT_PAID,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,CLAIM_AMOUNT_PREDICTION
0,213,66,432849,30-10-2001,CT,100/300,1546,2208,4146654,604147,...,No Police Report Available,71085.0,17655.0,1045.0,52385.0,Toyota,Camry,1999,No Fraud Reported,69999.179688
1,255,63,427352,17-04-2001,CT,100/300,1801,2501,3320595,613114,...,No Police Report Available,67762.0,474.0,4447.0,62841.0,Saab,92x,2004,No Fraud Reported,76015.187500
2,32,22,442056,13-10-1998,CT,100/300,1903,2642,8838774,602258,...,Unknown,51272.0,6672.0,1121.0,43479.0,Chevrolet,Malibu,2011,No Fraud Reported,48426.207031
3,11,36,465577,22-12-1992,CT,500/1000,1612,1941,9620213,432218,...,Police Report Available,92164.0,325.0,17178.0,74661.0,Audi,A5,2005,Fraud Reported,88199.375000
4,18,25,433126,05-03-2013,CT,250/500,2080,2418,9926398,605692,...,No Police Report Available,40113.0,400.0,925.0,38788.0,Saab,95,2000,No Fraud Reported,39471.875000


In [ ]:
# 2. Fraud Claim Decision Tree C